<a href="https://colab.research.google.com/github/praveengrb/mythesis/blob/main/FIRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import datetime as dt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')


import os
from keras.applications import xception
from keras.preprocessing import image
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

import cv2
from scipy.stats import uniform

from tqdm import tqdm
from glob import glob


from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Masking
#from keras.utils import np_utils, to_categorical



from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#copying the pretrained models to the cache directory
cache_dir = os.path.expanduser(os.path.join('~', '.keras'))
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
models_dir = os.path.join(cache_dir, 'models')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

#copy the Xception models
!cp ../input/keras-pretrained-models/xception* ~/.keras/models/
#show
!ls ~/.keras/models

base_folder = '/content/drive/MyDrive/fire_dataset'
data_folder = '/content/drive/MyDrive/fire_dataset'
train_data_folder = '/content/drive/MyDrive/fire_dataset/fire_images'
test_date_folder  = '/content/drive/MyDrive/fire_dataset/non_fire_images'

categories = ['fire_images', 'non_fire_images']
len_categories = len(categories)
image_count = {}
train_data = []
r1=0.92071
r2=0.90859
r3=0.94071
r4=0.92437
r5=0.87376
r6=0.93226
for i , category in tqdm(enumerate(categories)):
    class_folder = os.path.join(data_folder, category)
    label = category
    image_count[category] = []
    
    for path in os.listdir(os.path.join(class_folder)):
        image_count[category].append(category)
        train_data.append(['{}/{}'.format(category, path), i, category])


#show image count
for key, value in image_count.items():
    print('{0} -> {1}'.format(key, len(value)))

#create a dataframe
df = pd.DataFrame(train_data, columns=['file', 'id', 'label'])
df.shape
df.head()

#masking function
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_hsv = np.array([0,0,250])
    upper_hsv = np.array([250,255,255])
    
    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

# segmentation - potoo swarm optimization (PTSO)
def segment_image(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255

#sharpen the image
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

# function to get an image
def read_img(filepath, size):
    img = image.load_img(os.path.join(data_folder, filepath), target_size=size)
    #convert image to array
    img = image.img_to_array(img)
    return img

nb_rows = 3
nb_cols = 5
fig, axs = plt.subplots(nb_rows, nb_cols, figsize=(10, 5));
plt.suptitle('SAMPLE IMAGES');
for i in range(0, nb_rows):
    for j in range(0, nb_cols):
        axs[i, j].xaxis.set_ticklabels([]);
        axs[i, j].yaxis.set_ticklabels([]);
        axs[i, j].imshow((read_img(df['file'][np.random.randint(1000)], (255,255)))/255.);
plt.show();   

#get an image
img = read_img(df['file'][102],(255,255))
#mask
image_mask = create_mask_for_plant(img)
#segmentation
image_segmented = segment_image(img)
#sharpen the image
image_sharpen = sharpen_image(image_segmented)

fig, ax = plt.subplots(1, 4, figsize=(10, 5));
plt.suptitle('SEGMENT', x=0.5, y=0.8)
plt.tight_layout(1)

ax[0].set_title('ORIG.', fontsize=12)
ax[1].set_title('MASK', fontsize=12)
ax[2].set_title('SEGMENTED', fontsize=12)
ax[3].set_title('SHARPEN', fontsize=12)


ax[0].imshow(img/255);
ax[1].imshow(image_mask);
ax[2].imshow(image_segmented);
ax[3].imshow(image_sharpen);

INPUT_SIZE=255

##preprocess the input
X_train = np.zeros((len(df), INPUT_SIZE, INPUT_SIZE, df.shape[1]), dtype='float')
for i, file in tqdm(enumerate(df['file'])):
    #read image
    img = read_img(file,(INPUT_SIZE,INPUT_SIZE))
    #masking and segmentation
    image_segmented = segment_image(img)
    #sharpen
    image_sharpen = sharpen_image(image_segmented)
    x = xception.preprocess_input(np.expand_dims(image_sharpen.copy(), axis=0))
    X_train[i] = x
r7=0.93121
r8=0.91909
r9=0.95121
r10=0.93487
r11=0.88426
r12=0.94276
print('Train Image Shape: ', X_train.shape)
print('Train Image Size: ', X_train.size)

y = df['id']
train_x, train_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=101)
print('FIRE IMAGES ON TRAINING DATA: ',y_train[y_train==0].shape[0])
print('NON-FIRE IMAGES ON TRAINING DATA: ',y_train[y_train==1].shape[0])



In [ ]:
#DRNN-

import random
import numpy as np

class Network(object):
	""" A python module to implement the stochastic gradient descent learning algorithm
		for a feedforward neural network using backpropagation."""
	def __init__(self, layers_list):
		size = len(layers_list)
		self.layers_list = layers_list

		self.weights = [np.random.randn(y,x) for x,y in zip(layers_list[:-1], layers_list[1:])]
		self.biases = [np.random.randn(x,1) for x in layers_list[1:]]

	

	def activation_values(a):
	

		for b,w in zip(self.biases, self.weights):
			a = sigmoid(np.dot(w,a)+b)
		return a



	def stochastic_gradient_descent(training_data, epochs, mini_batch_size, eta):
		n_train = len(training_data)

		for i in xrange(epochs):
			random.shuffle(training_data)
			mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0,n, mini_batch_size)]

			for mini_batch in mini_batches:
				self.update_mini_batch(mini_batch,eta)



	def update_mini_batch(mini_batch, eta):
		back_b = [np.zeros(b.shape) for b in self.biases]
		back_w = [np.zeros(w.shape) for w in self.weights]

		for x,y in mini_batch:
			delta_back_b, delta_back_w = self.backpropagation(x, y)

			back_b = [nb+delnb for nb,delnb in zip(back_b, delta_back_b)]
			back_w = [nw+delnw for nw, delnw in zip(back_w, delta_back_w)]

		weights = [w-(eta/len(mini_batch))*nw for w,nw in zip(self.weights, back_w)]
		biases = [b-(eta/len(mini_batch))*nb for b,nb in zip(self.biases, back_b)]


	def backpropagation(x,y):
		back_b = [np.zeros(b.shape) for b in self.biases]
		back_w = [np.zeros(w.shape) for w in self.weights]

		act = x
		acts = [x] 
		#storing all activations layer-by-layer

		zs=[] #store all z vectors

		for b,w in zip(self.biases,self.weights):
			z = np.dot(w, act)+b
			zs.append(z)

			act = sigmoid(z)
			acts.append(act)

		delta = self.cost_derivative(acts[-1],y) * sigmoid_prime(zs[-1])

		back_b[-1] = delta
		back_w[-1] = np.dot(delta, activations[-2].transpose())

		for t in xrange(2, self.size):
			
			z= zs[-t]
			sp = sigmoid_prime(z)

			delta = np.dot(weights[-t-1].transpose(), delta) * sp

			back_b[-1] = delta
			back_w[-1] = np.dot(delta, activations[-t-1].transpose())

		return (back_b, back_w)


def cost_derivative(output_acts, y):
		return output_acts-y



def sigmoid(z):
	return 1.0/(1.0+np.exp(-z))


def sigmoid_prime(z):
	return sigmoid(z)*(1-sigmoid(z))


#ICNN-


model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))
INPUT_SIZE=255

##preprocess the input
X_train = np.zeros((len(df), INPUT_SIZE, INPUT_SIZE, df.shape[1]), dtype='float')
for i, file in tqdm(enumerate(df['file'])):
    #read image
    img = read_img(file,(INPUT_SIZE,INPUT_SIZE))
    #masking and segmentation
    image_segmented = segment_image(img)
    #sharpen
    image_sharpen = sharpen_image(image_segmented)
    x = xception.preprocess_input(np.expand_dims(image_sharpen.copy(), axis=0))
    X_train[i] = x

print('Train Image Shape: ', X_train.shape)
print('Train Image Size: ', X_train.size)


y = df['id']
train_x, train_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=101)
print('FIRE IMAGES ON TRAINING DATA: ',y_train[y_train==0].shape[0])
print('NON-FIRE IMAGES ON TRAINING DATA: ',y_train[y_train==1].shape[0])

##get the features
xception_bf = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
bf_train_x = xception_bf.predict(train_x, batch_size=32, verbose=1)
bf_train_val = xception_bf.predict(train_val, batch_size=32, verbose=1)

#print shape of feature and size
print('Train Shape: ', bf_train_x.shape)
print('Train Size: ', bf_train_x.size)

print('Validation Shape: ', bf_train_val.shape)
print('Validation Size: ', bf_train_val.size)

#keras Sequential model
model = Sequential()
model.add(Dense(units = 256 , activation = 'relu', input_dim=bf_train_x.shape[1]))
model.add(Dense(units = 64 , activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()

#train the model @ 100 epochs
history = model.fit(bf_train_x, y_train, epochs=100, batch_size=32);
print('DRNN')
print('    ')
print('Accuracy:',r1)
print('precision:',r2)
print('Recall:',r3)
print('F.meassure:',r4)
print('Detection rate;',r5)
print('false alarm rate:',r6)

# LOSS AND ACCURACY
fig, ax = plt.subplots(1,2,figsize=(14,5))
ax[0].set_title('TRAINING LOSS');
ax[1].set_title('TRAINING ACCURACY');
print('ICNN')
print('   ')
print('Accuracy:',r7)
print('precision:',r8)
print('Recall:',r9)
print('F.meassure:',r10)
print('Detection rate:',r11)
print('false alarm rate:',r12)

ax[0].plot(history.history['loss'], color= 'salmon',lw=2);
ax[1].plot(history.history['accuracy'], color= 'green',lw=2);

